In [1]:
import os
import numpy as np
import glob

import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dropout, Input
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
train_x = np.load('./train_375.npy')
test_x = np.load('./test_125.npy')

In [3]:
#train_a = np.zeros(shape=(1,61,61,275,1))
#for f in glob.glob("./train/*.npy"):
#    with open(f, "rb") as infile:
#            train_data = np.load(infile).reshape(1,61,61,275,1)
#            train_b = np.concatenate([train_data,train_a],axis=0)
#            train_a = train_b

In [4]:
#train_b.shape

In [5]:
#train_x = np.delete(train_b,375,0)

In [6]:
#np.save('./train_375.npy',train_x)

In [7]:
#test1 = np.zeros(shape=(1,61,61,275,1))
#for f in glob.glob("./test/*.npy"):
#    with open(f, "rb") as infile:
#            test = np.load(infile).reshape(1,61,61,275,1)
#            test2 = np.concatenate([test,test1],axis=0)
#            test1 = test2

In [8]:
test_x.shape

(125, 61, 61, 275, 1)

In [9]:
#test_x = np.delete(test2,125,0)

In [10]:
#np.save('./test_125.npy',test_x)

In [11]:
import pandas as pd

label_df = pd.read_csv("./germination_label.csv")
label_df.head(5)

,0
0,0
1,0
2,1
3,0
4,0


In [12]:
train_y = label_df[0:375]
train_y.head(5)

,0
0,0
1,0
2,1
3,0
4,0


In [13]:
from keras.utils import to_categorical
train_y = to_categorical(train_y)

In [14]:
test_y = label_df[375:500]
test_y.head(5)

,0
375,1
376,0
377,1
378,0
379,1


In [15]:
test_y = to_categorical(test_y)
test_y

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [16]:
input_layer = Input((61, 61, 275,1))

conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu')(conv_layer1)
conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(conv_layer2)
print(conv_layer3._keras_shape)
conv3d_shape = conv_layer3._keras_shape
conv_layer3 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer3)
conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer3)

flatten_layer = Flatten()(conv_layer4)

dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=2, activation='softmax')(dense_layer2)

Instructions for updating:
Colocations handled automatically by placer.
(None, 55, 55, 263, 32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
model = Model(inputs=input_layer, outputs=output_layer)

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 61, 61, 275, 1)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 59, 59, 269, 8)    512       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 57, 57, 265, 16)   5776      
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 55, 55, 263, 32)   13856     
_________________________________________________________________
reshape_1 (Reshape)          (None, 55, 55, 8416)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 64)        4847680   
_________________________________________________________________
flatten_1 (Flatten)          (None, 179776)            0         
__________

In [19]:
adam = Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [20]:
filepath = "best-model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [21]:
history = model.fit(x=train_x, y=train_y, batch_size=12, epochs=50, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
375/375 [==============================] - 43s 114ms/step - loss: 1.4278 - acc: 0.6853

Epoch 00001: acc improved from -inf to 0.68533, saving model to best-model.hdf5
Epoch 2/50
375/375 [==============================] - 34s 90ms/step - loss: 0.6133 - acc: 0.7147

Epoch 00002: acc improved from 0.68533 to 0.71467, saving model to best-model.hdf5
Epoch 3/50
375/375 [==============================] - 34s 91ms/step - loss: 0.5990 - acc: 0.7173

Epoch 00003: acc improved from 0.71467 to 0.71733, saving model to best-model.hdf5
Epoch 4/50
375/375 [==============================] - 34s 91ms/step - loss: 0.6035 - acc: 0.7173

Epoch 00004: acc improved from 0.71733 to 0.71733, saving model to best-model.hdf5
Epoch 5/50
375/375 [==============================] - 34s 92ms/step - loss: 0.5871 - acc: 0.7173

Epoch 00005: acc did not improve from 0.71733
Epoch 6/50
375/375 [==============================] - 35s 92ms/step - loss: 0.5749 - a

375/375 [==============================] - 35s 92ms/step - loss: 0.1098 - acc: 0.9360

Epoch 00049: acc did not improve from 0.93867
Epoch 50/50
375/375 [==============================] - 35s 92ms/step - loss: 0.1060 - acc: 0.9387

Epoch 00050: acc did not improve from 0.93867


In [22]:
# load best weights
model.load_weights("best-model.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [23]:
#model.predict(test_x,batch_size=10)
model.evaluate(test_x,test_y,batch_size=10)

125/125 [==============================] - 4s 36ms/step


[5.609525165557861, 0.49600000560283664]